In [2]:
import psycopg2
import pandas as pd

# Data Load

In [3]:
Fact_table=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Fact_table',engine='openpyxl')
Trans_dim=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Trans_dim',engine='openpyxl')
Item_dim=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Item_dim',engine='openpyxl')
Store_dim=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Store_dim',engine='openpyxl')
Time_dim=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Time_dim',engine='openpyxl')
Coustomer_dim=pd.read_excel(r'E:/NSU/1st Semester (Spring 2023)/CSE 512 (Distributed Database System)/Assignment/e-commerece_data.xlsx',sheet_name='Coustomer_dim',engine='openpyxl')

# Task 3

## Q1: Find the division/district/year/month wise total_sale_price joining fact table and respective dimension table 

In [19]:
q1a=Fact_table.merge(Store_dim, on='store_key')

q1a.drop(q1a.columns[[0,1,2,3,5,6,7,-1]], axis=1, inplace=True)
q1a=q1a.set_index(['division','district'])
q1a

store_key  total_price
division district                       
DHAKA    JAMALPUR    S00307        35.00
         JAMALPUR    S00307       253.00
         JAMALPUR    S00307       132.00
         JAMALPUR    S00307       108.00
         JAMALPUR    S00307        56.00
...                     ...          ...
RAJSHAHI NAOGAON     S00120        48.00
         NAOGAON     S00120        90.00
         NAOGAON     S00120        27.00
         NAOGAON     S00120        20.25
         NAOGAON     S00120       432.00

[1000000 rows x 2 columns]

In [23]:
q1b=Fact_table.merge(Time_dim, on='time_key')

q1b.drop(q1b.columns[[0,1,3,4,5,6,7,9,10,11,12,14]], axis=1, inplace=True)
q1b=q1b.set_index(['year','month'])
q1b

time_key  total_price
year month                      
2016 4      T049189        35.00
     4      T049189        30.00
     4      T049189        22.00
     4      T049189        84.00
     4      T049189       128.00
...             ...          ...
2015 3      T093351       128.00
2017 12     T044140       112.00
2016 9      T083509        18.00
2014 10     T070722        33.75
2020 6       T06156        40.50

[1000000 rows x 2 columns]

## Q2: Find the customer/bank/transaction(cash/online) wise total_sale_price joining fact table and respective dimension table 

In [27]:
q2a=Fact_table.merge(Coustomer_dim, on='coustomer_key')

q2a.drop(q2a.columns[[0,2,3,4,5,6,7,-2]], axis=1, inplace=True)
q2a=q2a.set_index(['nid','name'])
q2a

coustomer_key  total_price
nid           name                             
4543429591184 priya        C004510         35.0
              priya        C004510         99.0
              priya        C004510        140.0
              priya        C004510         27.0
              priya        C004510        225.0
...                            ...          ...
6147606639602 ravina       C004117         92.0
              ravina       C004117         56.0
              ravina       C004117        120.0
              ravina       C004117        280.0
              ravina       C004117         30.0

[1000000 rows x 2 columns]

In [30]:
q2b=Fact_table.merge(Trans_dim, on='payment_key')

q2b.drop(q2b.columns[[1,2,3,4,5,6,7]], axis=1, inplace=True)
q2b=q2b.set_index(['bank_name','trans_type'])
q2b

payment_key  total_price
bank_name            trans_type                         
Premier Bank Limited card              P026         35.0
                     card              P026        128.0
                     card              P026        100.0
                     card              P026        200.0
                     card              P026        320.0
...                                     ...          ...
Nagad                mobile            P039        318.0
                     mobile            P039        583.0
                     mobile            P039         65.0
                     mobile            P039         72.0
                     mobile            P039         92.0

[1000000 rows x 2 columns]

## Q3: Total sales in Barisal for item 'Pepsi - 12 oz cans'

In [83]:
q3=Fact_table.merge(Item_dim, on='item_key').merge(Store_dim, on='store_key')
q3.dtypes
filter_criteria=((q3['item_name']=='Pepsi - 12 oz cans') & (q3['division']=='BARISAL'))
print(q3.loc[filter_criteria,'quantity'].sum())

3384


## Q4: Total sales in 2015 for supplier 'BIGSO AB'

In [81]:
q4=Fact_table.merge(Item_dim, on='item_key').merge(Time_dim, on='time_key')
#print(q4)
q4.dtypes
filter_criteria=((q4['year']==2015) & (q4['supplier']=='BIGSO AB'))
print(filter_criteria)
print(q4.loc[filter_criteria,'quantity'].sum())

0         False
1         False
2         False
3         False
4         False
          ...  
999995    False
999996    False
999997    False
999998    False
999999    False
Length: 1000000, dtype: bool
85002


## Q5: Total sales of Dhaka in 2015

In [85]:
q5=Fact_table.merge(Time_dim, on='time_key').merge(Store_dim, on='store_key')
#print(q5)
q5.dtypes
filter_criteria=((q5['year']==2015) & (q5['division']=='DHAKA'))
y=q5.loc[filter_criteria,'quantity'].sum()
print(y)

333424


## Q6: Q6: For each store(item supplier), what are the top three products offered that are most often purchased?

In [96]:
q6=Fact_table.merge(Item_dim,on='item_key',suffixes=('_F','_D'))
q6=q6.set_index('item_key')
q6=q6.groupby(['item_name','supplier']).agg({'quantity':'count'})
q6=q6.sort_values(by='quantity',ascending=False)
q6=q6.drop(q6.index[3:])
q6

,,quantity
item_name,supplier,
Diet Pepsi - 12 oz cans,Indo Count Industries Ltd,3952
Nat.Valley PeanutButter Protein Bars,BIGSO AB,3939
100% Juice Box Variety 6.75 oz,BIGSO AB,3923


## Q7: What products have been sold through card or mobile since X days? [X=5 days]

In [140]:
q7=Fact_table.merge(Item_dim,on='item_key',suffixes=('_F','_iD')).merge(Trans_dim,on='payment_key').merge(Time_dim,on='time_key')
q7=q7.set_index('date')
q7=q7.sort_values(by='date',ascending=True)
filter_criteria=(((q7['trans_type']=='card') | (q7['trans_type']=='mobile')) & (q7['year']==2020) & (q7['month']==12))
Q7=q7[filter_criteria]
Q7=Q7.drop(['payment_key','nam_desc','unit_price_iD','man_country','coustomer_key','time_key','item_key','store_key','quantity','unit_F','unit_price_F','total_price','supplier','unit_iD','bank_name','hour','quarter'],axis=1)
filter_criteria1=(Q7['day']>=27)
Q7=Q7[filter_criteria1]
print(Q7)

                                                 item_name trans_type  day  \
date                                                                         
27-12-2020 00:10  Go-Gurt Low Fat Yogurt, Strwbry & Berry        card   27   
27-12-2020 00:10                  Sugar in the Raw Packets       card   27   
27-12-2020 00:10        Perrier Sparkling Water glass 11oz       card   27   
27-12-2020 00:10       Fiber One 90 Calorie Fudge Brownies       card   27   
27-12-2020 00:10            Extra Variety Packs Sugar Free       card   27   
...                                                    ...        ...  ...   
31-12-2020 23:01                Planters Nut Variety Packs       card   31   
31-12-2020 23:01                          Cherry Coke 12oz       card   31   
31-12-2020 23:01              San Pelligrino 16oz Plastic      mobile   31   
31-12-2020 23:01                        Pepsi - 12 oz cans       card   31   
31-12-2020 23:01          Paper Plate 10 inch Ultra Strong      

## Q8: What season(quarter) is the worst for each product item?

In [145]:
q8=Fact_table.merge(Item_dim,on='item_key',suffixes=('_F','_iD')).merge(Time_dim,on='time_key')
print(q8.dtypes)
q8=q8.groupby(['item_name','quarter']).agg({'quantity':'count'})
q8=q8.sort_values(by='quantity',ascending=False)
print(q8)

payment_key       object
coustomer_key     object
time_key          object
item_key          object
store_key         object
quantity           int64
unit_F            object
unit_price_F     float64
total_price      float64
item_name         object
nam_desc          object
unit_price_iD    float64
man_country       object
supplier          object
unit_iD           object
date              object
hour               int64
day                int64
week              object
month              int64
quarter           object
year               int64
dtype: object
                                                     quantity
item_name                                   quarter          
Pepsi - 12 oz cans                          Q1           2001
Muscle Milk Protein Shake Van. 11oz         Q1           1966
Sprite - 12 oz cans                         Q3           1958
Pepsi - 12 oz cans                          Q3           1922
Diet Coke - 12 oz cans                      Q3           1920
..

# Task 4

### 4. store and time dimensional inventory analytics

#### [The analysis is about finding the highest goods(unit) sold based on division and year using fact_table, store_dimension table and time_dimention table]

In [8]:
t4=Fact_table.merge(Time_dim,on='time_key').merge(Store_dim,on='store_key')
t4

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,date,hour,day,week,month,quarter,year,division,district,upazila
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,26-04-2016 17:13,17,26,4th Week,4,Q2,2016,DHAKA,JAMALPUR,ISLAMPUR
1,P007,C001198,T026950,I00255,S00307,6,pack,20.0,120.0,25-06-2019 15:36,15,25,4th Week,6,Q2,2019,DHAKA,JAMALPUR,ISLAMPUR
2,P034,C001530,T03431,I00163,S00307,11,bags,15.0,165.0,17-10-2015 00:28,0,17,3rd Week,10,Q4,2015,DHAKA,JAMALPUR,ISLAMPUR
3,P021,C002207,T086625,I00051,S00307,1,cans,23.0,23.0,15-12-2016 09:26,9,15,3rd Week,12,Q4,2016,DHAKA,JAMALPUR,ISLAMPUR
4,P002,C001506,T039330,I00224,S00307,4,oz,11.0,44.0,12-05-2017 06:32,6,12,2nd Week,5,Q2,2017,DHAKA,JAMALPUR,ISLAMPUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,P011,C002569,T092675,I00060,S00249,5,bottles,22.0,110.0,19-10-2015 08:53,8,19,3rd Week,10,Q4,2015,DHAKA,DHAKA,HAZARIBAGH
999996,P020,C000679,T012494,I00129,S00249,9,pk,18.0,162.0,23-06-2019 15:29,15,23,4th Week,6,Q2,2019,DHAKA,DHAKA,HAZARIBAGH
999997,P008,C003677,T092846,I00203,S00249,3,ct,18.0,54.0,20-09-2019 15:45,15,20,3rd Week,9,Q3,2019,DHAKA,DHAKA,HAZARIBAGH
999998,P026,C008412,T051369,I00102,S00249,4,tubs,16.0,64.0,28-11-2020 02:18,2,28,4th Week,11,Q4,2020,DHAKA,DHAKA,HAZARIBAGH


In [9]:
t4=t4.groupby(['year','division','unit']).agg({'quantity':'count'})
t4

quantity
year division unit              
2014 BARISAL  Bags           123
              Ct              64
              bags           474
              bars           203
              botlltes        38
...                          ...
2021 SYLHET   pack             6
              pk              15
              rolls            3
              tins             1
              tubs             7

[1004 rows x 1 columns]

In [13]:
t4=t4.sort_values(by=['year','quantity'],ascending=False)
t4

quantity
year division   unit              
2021 DHAKA      ct            1187
                cans           687
     CHITTAGONG ct             549
     DHAKA      bottles        433
     CHITTAGONG cans           336
...                            ...
2014 BARISAL    lb              35
     SYLHET     tins            33
                cartons         32
                botlltes        25
                oz.             24

[1004 rows x 1 columns]

#### We can see that each year based on which location what goods (unit) were sold in greater quantity. Such as in 2021 dhaka has 'ct' sold size of highest 1187 as well as in 2014 barisal has 'lb' sold size of highest 35 only. 